In [1]:
from monai import transforms, networks
import torch
from torch.nn.functional import softmax
from EfficientNetMultimodal import EfficientNetMultimodal
import json

ModuleNotFoundError: No module named 'monai'

In [3]:
from bimcv_aikit.models.classification import SwinViTMultimodal_v3

ModuleNotFoundError: No module named 'bimcv_aikit'

In [9]:
dir(bimcv_aikit.models)

AttributeError: module 'bimcv_aikit.models' has no attribute 'classification'

In [2]:
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=0)

In [6]:
device = 'cpu'

In [7]:
with open("/home/alejandro/Documents/CIPF/Chaimeleon/OpenChallenge/ProstateCancerRiskPrediction/input/psa-and-age.json") as fp:
    clinical_info = json.load(fp)
transform = transforms.Compose(
            [
                transforms.LoadImage(image_only=True),
                transforms.EnsureChannelFirst(channel_dim=None),
                transforms.Resize(
                    spatial_size=(128, 128, 32),
                    mode=("trilinear"),
                ),
                transforms.ScaleIntensity(minv=0.0, maxv=1.0),
                transforms.NormalizeIntensity(),
            ]
        )
#image = transform('/home/alejandro/Documents/CIPF/Chaimeleon/OpenChallenge/ProstateCancerRiskPrediction/input/images/axial-t2-prostate-mri/a55112de-6e99-4539-b587-5857729d6d51.mha')
image = transform('/home/alejandro/Documents/CIPF/Chaimeleon/OpenChallenge/ProstateCancerRiskPrediction/input/images/axial-t2-prostate-mri/a55112de-6e99-4539-b587-5857729d6d51.mha')
image = image.unsqueeze(0)
model = EfficientNetMultimodal(
    model_name="efficientnet-b7",
    n_classes=2,
    in_channels_eff=1,
    in_num_features=2,
)
model.load_state_dict(
    torch.load("model_best_weights.pth", map_location=device)["state_dict"]
)
model.to(device)
image = image.to(device)
clinical_info = torch.tensor(
    [clinical_info["patient_age"], clinical_info["psa"]], dtype=torch.float32
).to(device)


In [10]:
risk_score_likelihood = softmax(model(image,clinical_info.view(1,-1)),dim=1)[0][1].item()

In [12]:
clinical_info.view(1,-1)

tensor([[73.0000, 10.2800]])

In [11]:
risk_score_likelihood

0.24852226674556732

In [8]:
clinical_info


tensor([73.0000, 10.2800])

In [5]:
transform=monai.transforms.Compose(
        [
            monai.transforms.LoadImage(
                image_only=True, reader = "NibabelReader"
            ),
            monai.transforms.EnsureChannelFirst(channel_dim = None),
            monai.transforms.Resize(
                spatial_size=(256,256,30),
                mode=("trilinear"),
            ),
            monai.transforms.ScaleIntensity(minv=0.0, maxv=1.0),
            monai.transforms.NormalizeIntensity()
        ]
    )
image = transform('/home/alejandro/Documents/CIPF/Chaimeleon/OpenChallenge/ProstateCancerRiskPrediction/a55112de-6e99-4539-b587-5857729d6d51.mha')
image = image.unsqueeze(0)

In [6]:
image.shape

torch.Size([1, 1, 256, 256, 30])

In [8]:
risk_score_likelihood[0][1].item()

1.650426747801248e-05

In [16]:
image.shape

torch.Size([1, 1, 256, 256, 30])